# 3D Wannier calculations

In [1]:
# Directives on Jupyter
%reset
# Auto reload bootstrap.py
%load_ext autoreload
%autoreload 2

import numpy as np

In [ ]:
from wannier import *
import numpy as np
from positify import positify

N = 20
# V0 = 100
# ax = 820
# ay = 1000
R0 = np.array([3, 3, 7.2])
dvr = Wannier(
    N,
    R0,
    # lattice=np.array([2, 2], dtype=int),
    lattice=np.array([3, 3], dtype=int),
    # lc=(ax, ay),
    # trap=(2 * V0, 707, 2.17E3),
    # atom=86.9,
    # laser=707,
    # ascatt=95,
    dim=3,
    avg=1 / 2,
    sparse=True,
    symmetry=True)

E, W, parity = eigen_basis(dvr)

A, U = optimization(dvr, E, W, parity)
mu = np.diag(A)  # Diagonals are mu_i
t = -(A - np.diag(mu))  # Off-diagonals are t_ij
V = interaction(dvr, U, W, parity)
for i in range(dvr.bands):
    mu = np.diag(A[i])  # Diagonals are mu_i
    t = -(A[i] - np.diag(mu))  # Off-diagonals are t_ij
    print(f'mu_i,{i+1} = ', np.real(mu))
    print(f't_ij,{i+1} =', abs(t))
print('U_i =', abs(V))

| 1D | Chain (kHz) |
| -- | ----- |
| 2 | 0.33406693 |
| 3 | 0.29108221 0.29108221 |
| 4 | 0.29064946 0.24734239 0.29064945 |
| 5 | 2.90641293e-01 2.46824744e-01 2.46824745e-01 2.90641295e-01 |

| 2D | Plaquett (kHz) |
| -- | -------- |
| 2x2 | 0.33312028 0.09639722 |
| 3x3 | 2.90170850e-01 2.89149888e-01 9.02413012e-02 8.77313483e-02 |